In [46]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np
import torch

In [2]:
from netgen.occ import *

def BaseShape(l, w):
    wp1 = WorkPlane().RectangleC(l,w).Face()
    wp2 = WorkPlane().RectangleC(w,l).Face()
    cross = wp1 + wp2
    return cross


l = 0.3
w = 1
base = BaseShape(l, w)


In [3]:
offset = 3
figure = base
for i in range(3):
    for j in range(3):
        if (i+j) % 2 == 0:
            figure += BaseShape(i,j).Move((i*offset, j*offset, 0))
        else:
            if i+j == 1:
                figure += WorkPlane().Circle(1).Face().Move((i*offset, j*offset, 0))
            else:
                figure += base.Mirror(Axis((0, 0, 0), (1, 0, 0))
                      ).Move((i*offset, j*offset, 0))


geo = WorkPlane().Rectangle(5,5).Face() -  figure.Move((-2,-2,0))
Draw(geo)

geo.edges.Min(Y).name ='bottom'
geo.edges.Max(Y).name ='top'
geo.edges.Min(X).name ='left'
geo.edges.Max(X).name ='right'

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [4]:
Draw(geo, filename='./Test')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

BaseWebGuiScene

In [5]:



# shape = Glue([air])
# # Draw (shape);
# air = OCCGeometry(shape, dim=2)

# mesh = Mesh(air.GenerateMesh(maxh=0.1)).Curve(3)
# Draw (mesh);

In [6]:
plate = OCCGeometry(geo, dim=2)
mesh = Mesh(plate.GenerateMesh(maxh=0.1)).Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [39]:

# H1-conforming finite element space
fes = H1(mesh, order=1, dirichlet="bottom", autoupdate=True)
# define trial- and test-functions
u = fes.TrialFunction()
v = fes.TestFunction()

In [40]:
# the bilinear-form 
a = BilinearForm(grad(u)*grad(v)*dx)

funcf = 50*x*y
# funcf = 50*sin(y)
f = LinearForm(funcf*v*dx)

a.Assemble()
f.Assemble()
Draw(funcf,mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [41]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu);
# Draw(grad(gfu),mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [42]:
# for x in gfu.vec:
#     print(x)

In [48]:
sol =  torch.tensor([x for x in gfu.vec])

In [49]:
sol.shape

torch.Size([2195])

In [50]:

vertex_coordinates = []

for v in mesh.vertices:
    vertex_coordinates.append(v.point)

vertex_array = torch.tensor(np.array(vertex_coordinates))
print(vertex_array.shape)


torch.Size([2195, 2])
